# Import & Ingest

In [1]:
import sys
import pathlib
SOURCE_PATH = pathlib.Path.cwd().resolve().parent
sys.path.append(str(SOURCE_PATH))

In [2]:
from config import *
from utils import ingest_kaggle_database
import pandas as pd

In [3]:
ingest_kaggle_database(SOCCER_DATABASE)

In [4]:
%load_ext sql
%sql sqlite:///../data/database.sqlite
%config SqlMagic.style = "_deprecated_default"

In [5]:
%%sql
SELECT name FROM sqlite_schema
WHERE type = "table"
AND name NOT LIKE "sqlite%"
ORDER BY 1;

 * sqlite:///../data/database.sqlite
Done.


name
Country
League
Match
Player
Player_Attributes
Team
Team_Attributes


# Introductory Analysis & Validation Checks (Aiden)

In [6]:
%%sql
WITH test AS (SELECT * FROM Match WHERE home_team_api_id NOT NULL AND away_team_api_id NOT NULL)
SELECT COUNT(*) FROM test;

 * sqlite:///../data/database.sqlite
Done.


COUNT(*)
25979


In [7]:
%%sql
-- Count of matches in Match table
SELECT COUNT(*) FROM Match;

 * sqlite:///../data/database.sqlite
Done.


COUNT(*)
25979


In [8]:
%%sql
-- Count of unique teams
SELECT COUNT(DISTINCT team_long_name) AS unique_team_count
FROM Team;

 * sqlite:///../data/database.sqlite
Done.


unique_team_count
296


In [9]:
%%sql
-- Displays sample data of Team table
SELECT * FROM Team LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
1,9987,673,KRC Genk,GEN
2,9993,675,Beerschot AC,BAC
3,10000,15005,SV Zulte-Waregem,ZUL
4,9994,2007,Sporting Lokeren,LOK


In [10]:
%%sql
-- Displays sample data of Team_Attributes
SELECT * FROM Team_Attributes LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
1,434,9930,2010-02-22 00:00:00,60,Balanced,None,Little,50,Mixed,Organised,60,Normal,65,Normal,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
2,434,9930,2014-09-19 00:00:00,52,Balanced,48,Normal,56,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,434,9930,2015-09-10 00:00:00,47,Balanced,41,Normal,54,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
4,77,8485,2010-02-22 00:00:00,70,Fast,None,Little,70,Long,Organised,70,Risky,70,Lots,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover


In [11]:
%%sql
-- Displays sample data of Player_Attributes
SELECT * FROM Player_Attributes LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
1,218353,505942,2016-02-18 00:00:00,67,71,right,medium,medium,49,44,71,61,44,51,45,39,64,49,60,64,59,47,65,55,58,54,76,35,71,70,45,54,48,65,69,69,6,11,10,8,8
2,218353,505942,2015-11-19 00:00:00,67,71,right,medium,medium,49,44,71,61,44,51,45,39,64,49,60,64,59,47,65,55,58,54,76,35,71,70,45,54,48,65,69,69,6,11,10,8,8
3,218353,505942,2015-09-21 00:00:00,62,66,right,medium,medium,49,44,71,61,44,51,45,39,64,49,60,64,59,47,65,55,58,54,76,35,63,41,45,54,48,65,66,69,6,11,10,8,8
4,218353,505942,2015-03-20 00:00:00,61,65,right,medium,medium,48,43,70,60,43,50,44,38,63,48,60,64,59,46,65,54,58,54,76,34,62,40,44,53,47,62,63,66,5,10,9,7,7


In [12]:
%%sql
-- Count of unique players
SELECT COUNT(DISTINCT player_fifa_api_id) AS unique_players
FROM Player_Attributes;

 * sqlite:///../data/database.sqlite
Done.


unique_players
11062


In [13]:
%%sql
-- Top 25 rated players
SELECT overall_rating
FROM Player_Attributes
ORDER BY overall_rating DESC
LIMIT 25;

 * sqlite:///../data/database.sqlite
Done.


overall_rating
94
94
94
94
94
94
94
94
94
94


In [14]:
%%sql
-- What does the Country table look like?
SELECT * FROM Country LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,name
1,Belgium
1729,England
4769,France
7809,Germany


In [15]:
%%sql
-- What does the League table look like?
SELECT * FROM League LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,country_id,name
1,1,Belgium Jupiler League
1729,1729,England Premier League
4769,4769,France Ligue 1
7809,7809,Germany 1. Bundesliga


In [16]:
%%sql
-- What does the Match table look like?
-- Player data is incomplete!
SELECT * FROM Match LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.73,3.4,5,1.75,3.35,4.2,1.85,3.2,3.5,1.8,3.3,3.75,None,None,None,1.7,3.3,4.33,1.9,3.3,4,1.65,3.4,4.5,1.78,3.25,4,1.73,3.4,4.2
2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.95,3.2,3.6,1.8,3.3,3.95,1.9,3.2,3.5,1.9,3.2,3.5,None,None,None,1.83,3.3,3.6,1.95,3.3,3.8,2,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.6
3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.38,3.3,2.75,2.4,3.3,2.55,2.6,3.1,2.3,2.5,3.2,2.5,None,None,None,2.5,3.25,2.4,2.63,3.3,2.5,2.35,3.25,2.65,2.5,3.2,2.5,2.3,3.2,2.75
4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.44,3.75,7.5,1.4,4,6.8,1.4,3.9,6,1.44,3.6,6.5,None,None,None,1.44,3.75,6,1.44,4,7.5,1.45,3.75,6.5,1.5,3.75,5.5,1.44,3.75,6.5


In [17]:
%%sql
-- What leagues have player data?
SELECT DISTINCT 
    l.name AS league_name,
    c.name AS country_name
FROM League l
JOIN Country c ON c.id = l.country_id
JOIN Match m ON l.id = m.league_id
WHERE m.home_player_X1 IS NOT NULL
ORDER BY league_name;

 * sqlite:///../data/database.sqlite
Done.


league_name,country_name
Belgium Jupiler League,Belgium
England Premier League,England
France Ligue 1,France
Germany 1. Bundesliga,Germany
Italy Serie A,Italy
Netherlands Eredivisie,Netherlands
Poland Ekstraklasa,Poland
Portugal Liga ZON Sagres,Portugal
Scotland Premier League,Scotland
Spain LIGA BBVA,Spain


In [18]:
%%sql
-- What do these values such as "home_player_X5" mean?
SELECT * FROM Match WHERE home_player_X1 IS NOT NULL LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
145,1,1,2008/2009,24,2009-02-28 00:00:00,493016,9996,8635,1,1,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,2,4,6,8,4,6,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,7,10,10,39890,None,38788,38312,26235,None,None,None,26916,None,94289,34480,38388,26458,13423,38389,38798,30949,38253,106013,38383,46552,None,None,None,None,None,None,None,None,5.75,3.75,1.55,5.5,3.65,1.55,4.8,3.5,1.55,5,3.5,1.57,None,None,None,4.8,3.4,1.62,5.5,3.5,1.62,5,3.4,1.6,5.25,3.6,1.6,5.5,3.6,1.53
146,1,1,2008/2009,24,2009-02-27 00:00:00,493017,8203,9987,2,1,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,2,4,6,8,4,6,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,7,10,10,38327,67950,67958,67959,37112,36393,148286,67898,164352,38801,26502,37937,38293,148313,104411,148314,37202,43158,9307,42153,32690,38782,None,None,None,None,None,None,None,None,3,3.4,2.3,2.95,3.25,2.2,2.8,3.1,2.2,2.75,3.25,2.2,None,None,None,2.62,3.2,2.3,3.05,3.2,2.3,2.65,3.25,2.35,2.9,3.25,2.3,2.8,3.2,2.25
147,1,1,2008/2009,24,2009-02-28 00:00:00,493018,9986,9998,3,0,1,2,4,6,8,6,8,2,4,4,6,1,2,4,8,6,2,4,6,8,4,6,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,7,10,10,95597,None,None,38435,94462,46004,164732,None,38246,38423,38419,38252,39156,39151,166554,15652,39145,46890,38947,46881,39158,119118,None,None,None,None,None,None,None,None,2,3.4,3.4,1.8,3.35,4.05,1.8,3.2,3.7,1.9,3.2,3.5,None,None,None,None,None,None,1.83,3.4,4.1,1.9,3.25,3.75,1.83,3.25,4.25,1.91,3.25,3.6
148,1,1,2008/2009,24,2009-03-01 00:00:00,493019,9985,9984,4,0,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,2,4,6,8,4,6,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,7,10,10,None,39580,30692,37861,47411,119117,35412,39631,39591,25957,38369,36835,37047,37021,38186,27110,32863,37957,37909,104386,38251,37065,None,None,None,None,None,None,None,None,1.44,3.75,7.5,1.5,3.55,6.5,1.4,3.9,6,1.44,3.6,6.5,None,None,None,1.4,3.75,7,1.5,3.75,6.5,1.4,3.75,7.5,1.45,3.75,7.5,1.4,3.75,7.5


In [19]:
%%sql
-- Top 10 home scores
-- No outliers, scores look reasonable
SELECT home_team_goal, away_team_goal
FROM Match
ORDER BY home_team_goal DESC
LIMIT 10;

 * sqlite:///../data/database.sqlite
Done.


home_team_goal,away_team_goal
10,0
10,2
9,1
9,2
9,0
9,1
8,0
8,2
8,0
8,0


In [20]:
%%sql
-- Top 10 away scores
-- No outliers, scores look reasonable
SELECT home_team_goal, away_team_goal
FROM Match
ORDER BY away_team_goal DESC
LIMIT 10;

 * sqlite:///../data/database.sqlite
Done.


home_team_goal,away_team_goal
0,9
1,8
0,8
0,8
2,8
0,8
1,7
1,7
2,7
0,7


In [21]:
%%sql
-- What does the Player table look like?
SELECT * FROM Player LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198


# Exploratory Analysis & Visualization (Keano)

In [22]:
%%sql
-- Build Team Strength Ratings (Offense + Defense):
-- This compresses 40+ player stats into meaningful team metrics.
CREATE VIEW Team_Player_Strength AS
SELECT
m.match_api_id,
AVG(p.overall_rating) AS avg_rating,
AVG(p.finishing + p.shot_power + p.positioning) AS attack_strength,
AVG(p.interceptions + p.marking + p.standing_tackle + p.sliding_tackle) AS defense_strength,
AVG(p.gk_reflexes + p.gk_handling + p.gk_positioning) AS gk_strength
FROM Match m
JOIN Player_Attributes p
ON p.player_api_id IN (
m.home_player_1, m.home_player_2, m.home_player_3, m.home_player_4,
m.home_player_5, m.home_player_6, m.home_player_7, m.home_player_8,
m.home_player_9, m.home_player_10, m.home_player_11,
m.away_player_1, m.away_player_2, m.away_player_3, m.away_player_4,
m.away_player_5, m.away_player_6, m.away_player_7, m.away_player_8,
m.away_player_9, m.away_player_10, m.away_player_11
)
GROUP BY m.match_api_id;

 * sqlite:///../data/database.sqlite
Done.


[]

In [23]:
%%sql
SELECT * FROM Team_Player_Strength LIMIT 15;

 * sqlite:///../data/database.sqlite
Done.


match_api_id,avg_rating,attack_strength,defense_strength,gk_strength
483129,69.8857938718663,160.81337047353762,214.70056497175142,51.3008356545961
483130,72.94791666666667,181.0078125,233.5027027027027,50.013020833333336
483131,69.64885496183206,161.95038167938932,213.66803278688525,49.786259541984734
483132,71.91692307692308,174.14153846153846,231.2926045016077,46.55692307692308
483133,76.39606126914661,184.5426695842451,231.41137855579868,56.99781181619256
483134,73.20991253644316,157.2332361516035,229.60058309037902,58.08746355685131
483135,73.5530303030303,183.13131313131314,222.64961636828644,51.88383838383838
483136,74.7620320855615,175.57486631016042,218.2826086956522,58.75935828877005
483137,70.6654411764706,174.31617647058823,198.5408560311284,49.02205882352941
483138,73.19420289855073,178.06666666666666,228.23460410557186,52.43768115942029


In [24]:
%%sql
-- Best finishers:
SELECT team_api_id, shooting_quality
FROM Shot_Conversion_Power
ORDER BY shooting_quality DESC;

 * sqlite:///../data/database.sqlite
(sqlite3.OperationalError) no such table: Shot_Conversion_Power
[SQL: -- Best finishers:
SELECT team_api_id, shooting_quality
FROM Shot_Conversion_Power
ORDER BY shooting_quality DESC;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [25]:
%%sql
-- Defensive kings:
SELECT team_api_id, defense_resilience
FROM Defensive_Resilience
ORDER BY defense_resilience DESC
LIMIT 10;

 * sqlite:///../data/database.sqlite
(sqlite3.OperationalError) no such table: Defensive_Resilience
[SQL: -- Defensive kings:
SELECT team_api_id, defense_resilience
FROM Defensive_Resilience
ORDER BY defense_resilience DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [26]:
%%sql
-- Top attacking teams:
SELECT team_api_id, attack_efficiency
FROM Attacking_Efficiency
ORDER BY attack_efficiency DESC
LIMIT 10;

 * sqlite:///../data/database.sqlite
(sqlite3.OperationalError) no such table: Attacking_Efficiency
[SQL: -- Top attacking teams:
SELECT team_api_id, attack_efficiency
FROM Attacking_Efficiency
ORDER BY attack_efficiency DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [27]:
%%sql
-- Tactical Profile and Spider Chart:
SELECT
t.team_long_name,
AVG(ta.buildUpPlaySpeed) AS build_speed,
AVG(ta.chanceCreationShooting) AS shooting,
AVG(ta.chanceCreationPassing) AS passing,
AVG(ta.chanceCreationCrossing) AS crossing,
AVG(ta.defencePressure) AS pressure,
AVG(ta.defenceAggression) AS aggression
FROM Team_Attributes ta
JOIN Team t ON t.team_api_id = ta.team_api_id
WHERE t.team_long_name = 'FC Barcelona'
GROUP BY t.team_long_name;

 * sqlite:///../data/database.sqlite
Done.


team_long_name,build_speed,shooting,passing,crossing,pressure,aggression
FC Barcelona,35.833333333333336,53.0,45.166666666666664,33.333333333333336,64.33333333333333,54.5


In [28]:
%%sql
SELECT
home_team_api_id AS team_id,
COUNT(*) AS matches,
SUM(home_team_goal > away_team_goal) AS wins,
ROUND(100.0 * SUM(home_team_goal > away_team_goal) / COUNT(*), 2) AS win_rate
FROM Match
GROUP BY home_team_api_id
ORDER BY win_rate DESC
LIMIT 15;

 * sqlite:///../data/database.sqlite
Done.


team_id,matches,wins,win_rate
8634,152,131,86.18
8633,152,129,84.87
9773,124,102,82.26
9772,124,102,82.26
9823,136,109,80.15
9925,152,120,78.95
8640,136,105,77.21
10260,152,116,76.32
8593,136,103,75.74
8635,106,79,74.53


In [29]:
%%sql
-- All Team Win/Losses:
SELECT
t.team_long_name,
SUM(CASE
WHEN m.home_team_api_id = t.team_api_id AND m.home_team_goal > m.away_team_goal THEN 1
WHEN m.away_team_api_id = t.team_api_id AND m.away_team_goal > m.home_team_goal THEN 1
ELSE 0
END) AS wins,
SUM(CASE
WHEN m.home_team_api_id = t.team_api_id AND m.home_team_goal < m.away_team_goal THEN 1
WHEN m.away_team_api_id = t.team_api_id AND m.away_team_goal < m.home_team_goal THEN 1
ELSE 0
END) AS losses
FROM Team t
LEFT JOIN Match m ON t.team_api_id = m.home_team_api_id OR t.team_api_id = m.away_team_api_id
GROUP BY t.team_long_name
ORDER BY wins DESC;

 * sqlite:///../data/database.sqlite
Done.


team_long_name,wins,losses
FC Barcelona,234,27
Real Madrid CF,228,40
Celtic,218,36
FC Bayern Munich,193,35
Manchester United,192,55
Juventus,189,46
SL Benfica,185,27
FC Porto,183,23
Ajax,181,32
FC Basel,180,42
